In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
from torch import nn

from torchvision import datasets
from torchvision.transforms import transforms

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224)),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
])

train_img = datasets.CIFAR10(
    root = 'data',
    train = True,
    download = True,
    transform = transform,
)

test_img = datasets.CIFAR10(
    root = 'data',
    train = False,
    download = True,
    transform = transform,
)

100%|██████████| 170498071/170498071 [00:52<00:00, 3253920.82it/s]


Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


In [6]:
EPOCH = 10
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using Device:", DEVICE)

Using Device: cuda


In [7]:
train_loader = DataLoader(train_img, batch_size = BATCH_SIZE, shuffle = True)
test_loader = DataLoader(test_img, batch_size = BATCH_SIZE, shuffle = False)

In [28]:
cfg = {
    '16' : [64, 64, 'Pool', 128, 128, 'Pool', 256, 256, 256, 'Pool', 512, 512, 512, 'Pool',  512, 512, 512, 'Pool']
}

In [35]:
class VGG_net(nn.Module):
    def __init__(self,vgg_name, fc_h, fc_o, class_num):
        super(VGG_net,self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Sequential(
            nn.Linear(7*7*512, fc_h),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(fc_h, fc_o),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(fc_o, class_num),
            #nn.Softmax() crossentropy loss에 포함
        )

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3

        for x in cfg:
            if x == 'Pool':
                layers += [nn.MaxPool2d(kernel_size = 2, stride = 2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size = 3, padding = 1),
                          nn.BatchNorm2d(x),
                          nn.ReLU()]
                in_channels = x
        return nn.Sequential(*layers)

    def forward(self,x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [36]:
#VGG_net(layer_num, fc_h1, fc_h2, class_num)
model = VGG_net('16', 512, 256, 10 ).to(DEVICE)
print(model)

VGG_net(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding

In [37]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = LEARNING_RATE, momentum=0.9)

In [38]:
def train(train_loader, model, loss_fn, optimizer):
    model.train()
    
    size = len(train_loader.dataset)
    
    for batch, (X, y) in enumerate(train_loader):
        X, y = X.to(DEVICE), y.to(DEVICE)
        pred = model(X)

        # 손실 계산
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f}  [{current:>5d}]/{size:5d}')

In [39]:
def test(test_loader, model, loss_fn):
    model.eval()

    size = len(test_loader.dataset)
    num_batches = len(test_loader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:8f}\n")

In [40]:
for i in range(EPOCH) :
    print(f"Epoch {i+1} \n------------------------")
    train(train_loader, model, loss, optimizer)
    test(test_loader, model, loss)
print("Done!")

Epoch 1 
------------------------
loss: 2.293388  [    0]/50000
loss: 2.016072  [ 3200]/50000
loss: 1.912669  [ 6400]/50000
loss: 1.781562  [ 9600]/50000
loss: 1.460474  [12800]/50000
loss: 1.512012  [16000]/50000
loss: 1.536594  [19200]/50000
loss: 1.307445  [22400]/50000
loss: 1.660928  [25600]/50000
loss: 1.270063  [28800]/50000
loss: 0.997961  [32000]/50000
loss: 2.073931  [35200]/50000
loss: 1.428016  [38400]/50000
loss: 1.228362  [41600]/50000
loss: 1.256403  [44800]/50000
loss: 1.311202  [48000]/50000
Test Error: 
 Accuracy: 57.6%, Avg loss: 1.141851

Epoch 2 
------------------------
loss: 1.090933  [    0]/50000
loss: 1.165593  [ 3200]/50000
loss: 1.224466  [ 6400]/50000
loss: 0.883838  [ 9600]/50000
loss: 1.363288  [12800]/50000
loss: 1.422333  [16000]/50000
loss: 1.243636  [19200]/50000
loss: 0.914620  [22400]/50000
loss: 1.176242  [25600]/50000
loss: 0.983434  [28800]/50000
loss: 0.723485  [32000]/50000
loss: 1.042198  [35200]/50000
loss: 0.874794  [38400]/50000
loss: 1.386